In [9]:
import sys
import psycopg2 as pps
import pandas as pd

# postgresql to pandas dataframe
#   conn: psycopg2 conn
#   select_query: string, which is SQL command
#   column_names: list of string, column names
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
        #conn.commit() # no need to commit assume the select_query command is not updating DB
    except (Exception, pps.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # result from cursor: list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

#
# Main flow
#

# connect to postgresql
postgre_param_dic = {
    "host"      : "localhost",
    "database"  : "analysis",
    "user"      : "postgres",
    "password"  : "jackyen0213"
}

#conn = psycopg2.connect(database="analysis", user="postgres", password="jackyen0213", host="localhost", port="5432")
conn = None
try:
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database, DB name = "analysis"...')
    conn = pps.connect(**postgre_param_dic)
except (Exception, pps.DatabaseError) as error:
    print(error)
    sys.exit(1) 
print("Connection successful")

# postgresql to dataframe
column_names = ['id', 'first_name', 'last_name', 'school', 'hire_date', 'salary']
command = 'SELECT * FROM teachers;'
df = postgresql_to_dataframe(conn, select_query=command, column_names=column_names)
df.set_index(['id'], inplace=True)
print(df)

# close the connection
conn.close()


Connecting to the PostgreSQL database, DB name = "analysis"...
Connection successful
   first_name last_name               school   hire_date salary
id                                                             
1       Janet     Smith    F.D. Roosevelt HS  2011-10-30  36200
2         Lee  Reynolds    F.D. Roosevelt HS  1993-05-22  65000
3      Samuel      Cole  Myers Middle School  2005-08-01  43500
4    Samantha      Bush  Myers Middle School  2011-10-30  36200
5       Betty      Diaz  Myers Middle School  2005-08-30  43500
6    Kathleen     Roush    F.D. Roosevelt HS  2010-10-22  38500
